In [7]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity 
from tensorflow import keras
from tensorflow.keras.layers import Bidirectional,Dense,GlobalMaxPooling1D,LSTM,Embedding
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from sklearn.metrics import precision_score,recall_score

In [2]:
data = pd.read_csv("../../Data/clean_data.csv")

In [6]:
bert_preproccessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# Using a smaller model replace it with a bigger model
bert_model = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2")

In [8]:
def create_embedding(sentence):
    preroccessed_sentence = bert_preproccessor(sentence)
    bert_embedding = bert_model(preroccessed_sentence)
    return bert_embedding["sequence_output"]


In [9]:
X_train, X_test, y_train, y_test = train_test_split(data["Tweet Text"],data["Informativeness"])

In [10]:
# Defining the model

input = keras.Input(shape=(),dtype="string",name="input")

layer = Bidirectional(CuDNNLSTM(128,return_sequences=True))(create_embedding(input))
layer = Bidirectional(CuDNNLSTM(128,return_sequences=True))(layer)
layer = GlobalMaxPooling1D()(layer)
layer = Dense(512, activation="relu")(layer)
layer = Dense(1, activation="sigmoid")(layer)
model = keras.Model(inputs=[input],outputs=layer)


model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model.fit(X_train,y_train,epochs=5)
model.summary()

Epoch 1/5
24/24 [==============================] - 14s 116ms/step - loss: 0.3977 - accuracy: 0.8373
Epoch 2/5
24/24 [==============================] - 2s 101ms/step - loss: 0.2750 - accuracy: 0.8867
Epoch 3/5
24/24 [==============================] - 2s 94ms/step - loss: 0.2100 - accuracy: 0.9013
Epoch 4/5
24/24 [==============================] - 2s 94ms/step - loss: 0.1594 - accuracy: 0.9360
Epoch 5/5
24/24 [==============================] - 2s 94ms/step - loss: 0.1834 - accuracy: 0.9213
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'input_mask': (None 0           input[0][0]                    

In [11]:
model.evaluate(X_test,y_test,verbose=2)

8/8 - 2s - loss: 0.2735 - accuracy: 0.9120


[0.2734609544277191, 0.9120000004768372]

In [11]:
def convert_to_class(x):
    if(x > .5):
        return 1
    else:
        return 0


In [13]:
predictions = model.predict(X_test)
class_prediction = np.apply_along_axis(convert_to_class,1,predictions)
print(f"The recall of the model is {recall_score(y_test,class_prediction)}")
print(f"The precision of the model is {precision_score(y_test,class_prediction)}")

The recall of the model is 0.8645833333333334
The precision of the model is 0.8383838383838383
